In [3]:
# Setup -- from CS 231N GANs notebook
from __future__ import print_function, division
import tensorflow as tf
import numpy as np

import os
import nibabel as nib
from nibabel.testing import data_path


import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'



In [14]:
# A bunch of utility functions

def show_images(images):
    images = np.reshape(images, [images.shape[0], -1])  # images reshape to (batch_size, D)
    sqrtn = int(np.ceil(np.sqrt(images.shape[0])))
    sqrtimg = int(np.ceil(np.sqrt(images.shape[1])))

    fig = plt.figure(figsize=(sqrtn, sqrtn))
    gs = gridspec.GridSpec(sqrtn, sqrtn)
    gs.update(wspace=0.05, hspace=0.05)

    for i, img in enumerate(images):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(img.reshape([sqrtimg,sqrtimg]))
    return

def show_multimodal(image):
    # images = np.reshape(images, [images.shape[0], -1])  # images reshape to (batch_size, D)
    figdim = int(np.ceil(image.shape[0]/2))
    # sqrtimg = int(np.ceil(np.sqrt(images.shape[1])))
    
    dispInd = int(np.ceil(image.shape[3]/2))

    fig = plt.figure(figsize=(figdim, figdim))
    gs = gridspec.GridSpec(figdim, figdim)
    gs.update(wspace=0.05, hspace=0.05)

    for i in range(image.shape[0]):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(image[i,:,:,dispInd])
    return

def preprocess_img(x):
    return 2 * x - 1.0

def deprocess_img(x):
    return (x + 1.0) / 2.0

def rel_error(x,y):
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

def count_params():
    """Count the number of parameters in the current TensorFlow graph """
    param_count = np.sum([np.prod(x.get_shape().as_list()) for x in tf.global_variables()])
    return param_count


def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    return session

In [62]:
# Open 
path = '../HGG/Brats17_2013_2_1/Brats17_2013_2_1_flair.nii'
flair_img = nib.load(path).get_data()

path = '../HGG/Brats17_2013_2_1/Brats17_2013_2_1_t1.nii'
t1_img = nib.load(path).get_data()

path = '../HGG/Brats17_2013_2_1/Brats17_2013_2_1_t2.nii'
t2_img = nib.load(path).get_data()

path = '../HGG/Brats17_2013_2_1/Brats17_2013_2_1_t1ce.nii'
t1ce_img = nib.load(path).get_data()

path = '../HGG/Brats17_2013_2_1/Brats17_2013_2_1_seg.nii'
seg_img = nib.load(path).get_data()

# img = np.zeros((4, 240, 240, 155))
# img[0,:,:,:] = flair_img
# img[1,:,:,:] = t1_img
# img[2,:,:,:] = t2_img
# img[3,:,:,:] = t1ce_img

# show_multimodal(img)

0


In [68]:
files[-1][:-6]

'Brats17_2013_10_1_'

In [72]:

i = 0
for root, dirs, files in os.walk('../HGG/'):
    if len(files) > 1:
        HGGnp = np.zeros((4,240,240,155))

        flair_img = nib.load(root + '/' + files[-5]).get_data()
        HGGnp[0,:,:,:] = flair_img
        
        seg = nib.load(root + '/' + files[-4]).get_data()
        HGGsegnp = seg
        
        t1_img = nib.load(root + '/' + files[-3]).get_data()
        HGGnp[1,:,:,:] = flair_img
        
        t1ce_img = nib.load(root + '/' + files[-2]).get_data()
        HGGnp[2,:,:,:] = flair_img
        
        t2_img = nib.load(root + '/' + files[-1]).get_data()
        HGGnp[3,:,:,:] = flair_img
        
        np.savez_compressed(root + '/' + files[-1][:-7], img = HGGnp, seg = HGGsegnp)
        
        i +=1
        print(i)


# HGG = tf.Variable(tf.stack(HGGnp), name="HGG", trainable=False)

i = 0
for root, dirs, files in os.walk('../LGG/'):
    if len(files) > 1:
        LGGnp = np.zeros((4,240,240,155))

        flair_img = nib.load(root + '/' + files[-5]).get_data()
        LGGnp[0,:,:,:] = flair_img
        
        seg = nib.load(root + '/' + files[-4]).get_data()
        LGGsegnp = seg
        
        t1_img = nib.load(root + '/' + files[-3]).get_data()
        LGGnp[1,:,:,:] = flair_img
        
        t1ce_img = nib.load(root + '/' + files[-2]).get_data()
        LGGnp[2,:,:,:] = flair_img
        
        t2_img = nib.load(root + '/' + files[-1]).get_data()
        LGGnp[3,:,:,:] = flair_img
        
        np.savez_compressed(root + '/' + files[-1][:-7], img = LGGnp, seg = LGGsegnp)
        
        i +=1
        print(i)

# LGG = tf.Variable(tf.zeros([75,4,240,240,155]), name="LGG", trainable=False)

        
        

1


KeyboardInterrupt: 

In [57]:
np.save('HGG.npy',HGGnp)

KeyboardInterrupt: 

29
